In [1]:
from Glucose_processor import GlucoseDataProcessor
from blood_glucose_predictor import BloodGlucosePredictor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [16]:
Glucose_data = GlucoseDataProcessor(CGM_file_path="CGM_data.csv", bolus_file_path="Bolus_data.csv", carb_file_path="Carb_data.csv",
                                    min_range=69, max_range=180, target=85, tp=75, td=280, dp=45, dd=180)


df = Glucose_data.create_filtered_DataFrame()

# Initialize the predictor
horizons = [5, 10, 15, 20, 30, 60]
predictor = BloodGlucosePredictor(horizons)

# Preprocess data (including handling missing values with interpolation)
X, y = predictor.preprocess_data(df)

df

d:\DiaPulse\blood_glucose_predictor.py:21: FutureWarning:

DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.



,Time,CGM,Insulin Activity,Food Activity,hour,minute,second,hour_fraction,hour_sin,hour_cos,BG_t+5,BG_t+10,BG_t+15,BG_t+20,BG_t+30,BG_t+60
3255,2024-12-09 01:57:16,74.633941,0.000000,0.000000,1,57,16,0.081435,0.489636,0.871927,77.013161,79.895722,83.361943,87.477852,97.802204,141.931251
3254,2024-12-09 02:02:16,77.013161,0.000000,0.000000,2,2,16,0.084907,0.508541,0.861038,79.895722,83.361943,87.477852,92.286805,104.002860,149.747136
3253,2024-12-09 02:07:16,79.895722,0.000000,0.000000,2,7,16,0.088380,0.527203,0.849739,83.361943,87.477852,92.286805,97.802204,110.830518,157.142641
3252,2024-12-09 02:12:16,83.361943,0.000000,0.000000,2,12,16,0.091852,0.545615,0.838036,87.477852,92.286805,97.802204,104.002860,118.188673,163.892997
3251,2024-12-09 02:17:16,87.477852,0.000000,0.000000,2,17,16,0.095324,0.563766,0.825934,92.286805,97.802204,104.002860,110.830518,125.942857,169.790115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,2024-12-20 12:51:29,109.909232,0.027243,0.137798,12,51,29,0.535752,-0.222754,-0.974875,112.474523,115.372253,118.539996,121.911326,128.991958,149.673834
15,2024-12-20 12:56:28,112.474523,0.029130,0.145712,12,56,28,0.539213,-0.243897,-0.969801,115.372253,118.539996,121.911326,125.417160,132.581176,152.754635
14,2024-12-20 13:01:28,115.372253,0.030675,0.151448,13,1,28,0.542685,-0.264995,-0.964250,118.539996,121.911326,125.417160,128.991958,136.144884,155.653519
13,2024-12-20 13:06:27,118.539996,0.031900,0.155199,13,6,27,0.546146,-0.285897,-0.958260,121.911326,125.417160,128.991958,132.581176,139.656314,158.336854


In [17]:
X

array([[0.74482366, 0.93597281, 0.12469817, 0.        , 0.        ],
       [0.75427609, 0.93052827, 0.13181539, 0.        , 0.        ],
       [0.76360751, 0.92487882, 0.14043831, 0.        , 0.        ],
       ...,
       [0.36749975, 0.01786527, 0.24656313, 0.49885655, 0.54377471],
       [0.35704839, 0.02086009, 0.25603914, 0.51878209, 0.557242  ],
       [0.34663001, 0.02409258, 0.26612417, 0.5341104 , 0.56443417]])

In [26]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(np.shape(X_train))

(2595, 5)


In [20]:
# Build and train the model
predictor.build_model(input_shape=(1, X_train.shape[1]))
predictor.train(X_train, y_train, epochs=500)

# Evaluate the model
loss, mae = predictor.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test MAE: {mae:.4f}')

# Make predictions
predictions = predictor.predict(X_test)
print(f"Predicted: {predictions[0]}, Actual: {y_test[0]}")

Epoch 1/500


d:\DiaPulse\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.1188 - mae: 0.2905 - val_loss: 0.0398 - val_mae: 0.1427
Epoch 2/500
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0322 - mae: 0.1312 - val_loss: 0.0209 - val_mae: 0.1097
Epoch 3/500
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0197 - mae: 0.1061 - val_loss: 0.0140 - val_mae: 0.0883
Epoch 4/500
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0110 - mae: 0.0788 - val_loss: 0.0077 - val_mae: 0.0634
Epoch 5/500
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0070 - mae: 0.0593 - val_loss: 0.0051 - val_mae: 0.0484
Epoch 6/500
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0049 - mae: 0.0475 - val_loss: 0.0045 - val_mae: 0.0436
Epoch 7/500
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0043 - mae: 0.0439 - val_loss: 0.0043 - val_mae: 0.0421
Epoch 8/500
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0042 - mae: 0.0425 - val_loss: 0.0042 - val_mae: 0.0413
Epoch 9/500
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0042 - mae: 0.0

In [23]:
# Make predictions
predictions = predictor.predict(X_test)
print(f"Predicted: {predictions}, Actual: {predictor.scaler_y.inverse_transform(y_test)}")

import plotly.express as px
p=[]
for i in predictions:
    p.append(i[0])

act=[]
for i in predictor.scaler_y.inverse_transform(y_test):
    act.append(i[0])
# Plot the predictions
fig = px.scatter(x=act, y=p)
fig.show()

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Predicted: [[115.44935  115.319    115.98885  116.95305  116.40472  117.89349 ]
 [139.91536  141.0459   141.40031  141.16113  141.71176  142.61354 ]
 [124.502655 125.24387  126.26749  125.79469  127.12506  131.82446 ]
 ...
 [281.3754   282.41467  283.84158  282.77094  281.87305  262.08292 ]
 [147.06761  146.37549  146.28012  145.83815  143.4958   135.5969  ]
 [117.72723  119.21331  120.71923  122.981995 125.14907  138.668   ]], Actual: [[115.68099018 116.11647811 116.43210122 116.6360243  116.75265081
  115.6457738 ]
 [141.90096235 143.96009482 145.31242341 145.93436273 145.04036859
  131.21688567]
 [120.04030854 115.26705186 110.60472831 106.05829888  97.36183889
   76.08331765]
 ...
 [279.44750953 278.99487317 278.43474277 277.78022599 276.24344591
  269.35088175]
 [147.31933049 146.81246292 146.33045252 145.82746848 144.61012603
  137.9299182 ]
 [114.34248679 110.46610889 106.50859019 102.540643    94.78172624
   74.08035774]]


In [30]:
# Look at the fit on the training data
predictions = predictor.predict(X_train)
print(f"Predicted: {predictions}, Actual: {predictor.scaler_y.inverse_transform(y_train)}")

import plotly.express as px
p=[]
for i in predictions:
    p.append(i[-1])

act=[]
for i in predictor.scaler_y.inverse_transform(y_train):
    act.append(i[-1])
# Plot the predictions
fig = px.scatter(x=act, y=p)
fig.show()

82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Predicted: [[125.01215  125.61927  122.60326  124.99841  127.26168  130.2414  ]
 [202.7782   202.72603  200.28824  199.30144  196.44275  184.17728 ]
 [130.98825  131.51624  129.87645  128.85638  129.80313  122.452255]
 ...
 [ 87.307625  87.5939    89.90664   92.63971   94.67555  113.21167 ]
 [113.384514 113.62282  114.62325  115.983215 115.938126 120.03447 ]
 [119.524765 118.83352  118.49941  118.28834  117.43847  113.48236 ]], Actual: [[116.97685441 110.87062994 105.17530672  99.93232707  90.90653936
   76.68645267]
 [200.34126483 195.2741952  190.28796458 185.40903146 176.01475281
  150.2538493 ]
 [131.63562655 129.45047851 126.87068886 123.93452154 117.15100085
   92.35599475]
 ...
 [ 83.95172316  80.56180879  77.26991762  74.08035774  68.10060601
   56.22999194]
 [116.45368173 120.88836532 125.71149708 130.88337925 142.02493417
  174.26926702]
 [121.62647405 122.64335844 123.82532479 125.12317967 127.87648413
  133.88764085]]
